# Core 10 — Survivability (Min-Window, Operational Relaxation)

본 노트북은 Core10의 **운영 완화 보조 지표**를 생성한다.

- Core10_03의 `y_survive_T` 계약은 유지한다.
- 본 노트북은 **관측 가능한 만큼만 평가하는 보조 지표**
  `y_survive_min_T` 를 생성한다.

정의 요약:

- survivability = "추가 설계 없이 운영 가능한 상태"
- 관측 길이가 T보다 짧아도,
  - 가능한 구간(H_eff)에 대해
  - SoMS 악화 속도 + 재진입 여부만 평가

⚠️ 본 지표는 **계약 타깃이 아님**
→ 운영 분석 / 보조 증거 / 해석용

In [9]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

CANDIDATES = [
    Path("../artifact/core10/core10_03_survivability_targets_preview.csv")
]

POST_PATH = None
for p in CANDIDATES:
    if p.exists():
        POST_PATH = p
        break

if POST_PATH is None:
    raise FileNotFoundError("Run core10_03 first: survivability preview not found")

print("Using POST_PATH:", POST_PATH.resolve())

post = pd.read_csv(POST_PATH)
print("loaded rows:", len(post))
post.head()

RULE_ID = "core10_03b_survivability_min_window_v1"

HORIZONS = [10, 20]

# step당 허용 SoMS 증가량
SOMS_RATE_MAX = 0.5

REENTRY_COL = "reentry_flag"

EXPORT_DIR = Path("../artifact/core10")
EXPORT_DIR.mkdir(exist_ok=True)

OUT_PREVIEW  = EXPORT_DIR / "core10_03b_survivability_min_targets.csv"
OUT_CONTRACT = EXPORT_DIR / "core10_03b_survivability_min_contract.json"

Using POST_PATH: /Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_03_survivability_targets_preview.csv
loaded rows: 9


In [3]:
lens = post.groupby(["run_id","case_id","antibody_id"]).size()

print(lens.describe())
print("len>=10:", (lens >= 10).mean())
print("len>=20:", (lens >= 20).mean())

def compute_survivability_min_group(g, H, SOMS_RATE_MAX):
    """
    g: single (run_id, case_id, antibody_id) trajectory
    returns: pd.Series (index = g.index)
    """
    steps   = g["step"].values
    soms    = g["SoMS_cumsum_window"].values
    reentry = g[REENTRY_COL].values

    y = np.zeros(len(g), dtype=int)
    max_step = np.nanmax(steps)

    for i, t in enumerate(steps):
        if pd.isna(t):
            continue

        t = int(t)
        available = max_step - t
        H_eff = min(H, available)

        if H_eff <= 0:
            continue

        target_step = t + H_eff
        idx = np.where(steps == target_step)[0]

        if len(idx) == 0:
            continue

        j = idx[0]

        delta = soms[j] - soms[i]
        rate  = delta / H_eff

        ok_soms = (not np.isnan(rate)) and (rate <= SOMS_RATE_MAX)
        ok_reentry = (reentry[i+1:j+1].max() == 0) if j > i else True

        y[i] = int(ok_soms and ok_reentry)

    return pd.Series(y, index=g.index)

count    1.0
mean     9.0
std      NaN
min      9.0
25%      9.0
50%      9.0
75%      9.0
max      9.0
dtype: float64
len>=10: 0.0
len>=20: 0.0


In [4]:
for H in HORIZONS:
    col = f"y_survive_min_{H}"

    parts = []
    for _, g in post.groupby(["run_id","case_id","antibody_id"], sort=False):
        s = compute_survivability_min_group(g, H, SOMS_RATE_MAX)
        parts.append(s)

    # 🔑 완전히 row-aligned 된 단일 Series
    y = pd.concat(parts).sort_index()
    y.name = col

    post[col] = y

In [5]:
post[[
    "run_id","case_id","antibody_id","step",
    "SoMS_cumsum_window","reentry_flag",
    "y_survive_10","y_survive_min_10"
]].head(20)

post["y_survive_10"].mean(), post["y_survive_min_10"].mean()

(0.0, 0.0)

In [6]:
for H in HORIZONS:
    col = f"y_survive_min_{H}"
    print(col, "mean:", post[col].mean())

if "case_id" in post.columns:
    post["bucket"] = post["case_id"].astype(str).map(
        lambda x: "A_ALWAYS_ALLOW" if x.startswith("A_")
        else ("B_GOVERNED" if x.startswith("B_") else "OTHER")
    )

    for H in HORIZONS:
        col = f"y_survive_min_{H}"
        print("\n", col)
        print(post.groupby("bucket")[col].mean())

y_survive_min_10 mean: 0.0
y_survive_min_20 mean: 0.0

 y_survive_min_10
bucket
B_GOVERNED    0.0
Name: y_survive_min_10, dtype: float64

 y_survive_min_20
bucket
B_GOVERNED    0.0
Name: y_survive_min_20, dtype: float64


In [10]:
post.to_csv(OUT_PREVIEW, index=False)
print("Exported:", OUT_PREVIEW)

Exported: ../artifact/core10/core10_03b_survivability_min_targets.csv


In [11]:
contract = {
    "contract_id": RULE_ID,
    "type": "operational_relaxation_metric",
    "parent_contract": "core10_03_survivability_contract",
    "definition": {
        "metric": "y_survive_min_H",
        "window": "H_eff = min(H, available_steps)",
        "soms_rule": {
            "metric": "SoMS_cumsum_window",
            "rate_definition": "(SoMS(t+H_eff)-SoMS(t))/H_eff",
            "rate_max": SOMS_RATE_MAX
        },
        "reentry_rule": "No re-entry within (t, t+H_eff]"
    },
    "notes": [
        "This is NOT a contractual shutdown criterion.",
        "Used for operational analysis when post-shutdown data is short.",
        "Preserves Core10 contract integrity."
    ]
}

OUT_CONTRACT.write_text(json.dumps(contract, indent=2), encoding="utf-8")
print("Exported:", OUT_CONTRACT)

Exported: ../artifact/core10/core10_03b_survivability_min_contract.json
